# (12.1) Установка всех необходимых пакетов¶

In [1]:
import numpy as np
import pandas as pd
import datetime as dt

# (12.2) Определение функций¶

In [2]:
def ReverseToDateTime(t):
    format = '%Y-%m-%d %H:%M:%S.%f'
    time_str = dt.datetime.strptime(t,format)
    return time_str

# (12.3) Список всех используемых датафреймов

In [ ]:
# iceberg_all - список всех найденных айсбергов
# groups - результаты группировки айсберг-заявок по дате, инструменту, направлению, цене, объему видимой части
# groups_1 - список элементов из groups, в состав которых вошло более одного элемента
# a - датафрейм на основании iceberg_all, в котором добавлено поле GROUP_ID - идентификатор цепочки, в который вошла айсберг-заявка
# for_big_ice - результаты группировки айсберг-заявок по идентификатору цепочки айсберг-заявок
# big_ice - список цепочек из for_big_ice, в состав которых вошло более одной айсберг-заявки
# b - датафрейм b на основании a, в который будут добавлены новые поля: TOTAL_VOLUME - общий объем цепочки и
# VOLUME_LEFT - объем, который остался в айсберге после вскрытия текущего элемента цепочки айсберга

# (12.4) Чтение данных

In [3]:
# читаем данные по найденным айсбергам
# iceberg_all - список всех найденных айсбергов
iceberg_all = pd.read_csv('iceberg.csv', header=0)

iceberg_all

,index,BUYSELL,DATE,ICEBERG,ORDERNO,PRICE,SECCODE,TIME,TIME_IDENT,VOLUME,VOLUME_INI,DELTA
0,0,B,20190304,1,199077,156.28,GAZP,2019-03-04 10:06:43.595004,2019-03-04 10:09:36.467866,-2830,2000,0 days 00:02:52.872862000
1,1,B,20190304,1,359638,156.18,GAZP,2019-03-04 10:30:27.072289,2019-03-04 10:53:31.759856,-210,2000,0 days 00:23:04.687567000
2,2,S,20190304,1,397852,156.58,GAZP,2019-03-04 10:36:57.193855,2019-03-04 10:38:05.393784,-3000,2000,0 days 00:01:08.199929000
3,3,B,20190304,1,486370,156.18,GAZP,2019-03-04 10:53:31.809856,2019-03-04 10:53:42.904115,-1000,1790,0 days 00:00:11.094259000
4,4,B,20190304,1,516105,156.03,GAZP,2019-03-04 11:00:03.906212,2019-03-04 11:00:05.256841,-1000,2000,0 days 00:00:01.350629000
...,...,...,...,...,...,...,...,...,...,...,...,...
54501,436,B,20190329,1,3821455,92.47,ALRS,2019-03-29 18:32:19.963550,2019-03-29 18:32:19.963586,-360,930,0 days 00:00:00.000036000
54502,437,B,20190329,1,3845898,92.58,ALRS,2019-03-29 18:35:14.255473,2019-03-29 18:35:14.256529,-1000,1000,0 days 00:00:00.001056000
54503,438,B,20190329,1,3845899,92.58,ALRS,2019-03-29 18:35:14.255473,2019-03-29 18:35:14.256529,-490,1000,0 days 00:00:00.001056000
54504,439,B,20190329,1,3845904,92.58,ALRS,2019-03-29 18:35:14.257134,2019-03-29 18:35:14.257134,-770,510,0 days 00:00:00.000000000


In [4]:
# проверяем типы данных переменных
iceberg_all.dtypes

index           int64
BUYSELL        object
DATE            int64
ICEBERG         int64
ORDERNO         int64
PRICE         float64
SECCODE        object
TIME           object
TIME_IDENT     object
VOLUME          int64
VOLUME_INI      int64
DELTA          object
dtype: object

In [5]:
# меняем тип данных у времени
iceberg_all['TIME'] = [ReverseToDateTime(iceberg_all.TIME[i]) for i in range(len(iceberg_all.TIME))]
iceberg_all['TIME_IDENT'] = [ReverseToDateTime(iceberg_all.TIME_IDENT[i]) for i in range(len(iceberg_all.TIME_IDENT))]

In [6]:
# заменяем столбец индекс на айди
iceberg_all['ID'] = [i for i in range(len(iceberg_all))]
del iceberg_all['index']

iceberg_all

,BUYSELL,DATE,ICEBERG,ORDERNO,PRICE,SECCODE,TIME,TIME_IDENT,VOLUME,VOLUME_INI,DELTA,ID
0,B,20190304,1,199077,156.28,GAZP,2019-03-04 10:06:43.595004,2019-03-04 10:09:36.467866,-2830,2000,0 days 00:02:52.872862000,0
1,B,20190304,1,359638,156.18,GAZP,2019-03-04 10:30:27.072289,2019-03-04 10:53:31.759856,-210,2000,0 days 00:23:04.687567000,1
2,S,20190304,1,397852,156.58,GAZP,2019-03-04 10:36:57.193855,2019-03-04 10:38:05.393784,-3000,2000,0 days 00:01:08.199929000,2
3,B,20190304,1,486370,156.18,GAZP,2019-03-04 10:53:31.809856,2019-03-04 10:53:42.904115,-1000,1790,0 days 00:00:11.094259000,3
4,B,20190304,1,516105,156.03,GAZP,2019-03-04 11:00:03.906212,2019-03-04 11:00:05.256841,-1000,2000,0 days 00:00:01.350629000,4
...,...,...,...,...,...,...,...,...,...,...,...,...
54501,B,20190329,1,3821455,92.47,ALRS,2019-03-29 18:32:19.963550,2019-03-29 18:32:19.963586,-360,930,0 days 00:00:00.000036000,54501
54502,B,20190329,1,3845898,92.58,ALRS,2019-03-29 18:35:14.255473,2019-03-29 18:35:14.256529,-1000,1000,0 days 00:00:00.001056000,54502
54503,B,20190329,1,3845899,92.58,ALRS,2019-03-29 18:35:14.255473,2019-03-29 18:35:14.256529,-490,1000,0 days 00:00:00.001056000,54503
54504,B,20190329,1,3845904,92.58,ALRS,2019-03-29 18:35:14.257134,2019-03-29 18:35:14.257134,-770,510,0 days 00:00:00.000000000,54504


# (12.5) Группировка

In [7]:
%%time
# группируем айсберг-заявки по параметрам: дата (DATE), инструмент (SECCODE), направление (BUYSELL), цена (PRICE), 
# объем видимой части (VOLUME INI)
# groups - результаты группировки айсберг-заявок по дате, инструменту, направлению, цене, объему видимой части
groups = iceberg_all.groupby(['DATE','SECCODE','BUYSELL','PRICE','VOLUME_INI'])['ID'].agg(lambda x: x.unique().tolist())

groups

CPU times: user 8.35 s, sys: 74.7 ms, total: 8.43 s
Wall time: 8.37 s


DATE      SECCODE  BUYSELL  PRICE   VOLUME_INI
20190304  ALRS     B        93.5    40                                                       [2060]
                                    50                                                       [2015]
                                    180                                                      [2145]
                                    190                                                      [2039]
                                    200                                                      [2014]
                                                                        ...                        
20190329  YNDX     S        2320.0  490                                                     [53495]
                                    491                                                     [53486]
                                    498                                                     [53477]
                                    500           [53

In [8]:
# преобразуем в датафрейм
groups = pd.DataFrame(groups)

groups

ID
DATE     SECCODE BUYSELL PRICE  VOLUME_INI                                                   
20190304 ALRS    B       93.5   40                                                     [2060]
                                50                                                     [2015]
                                180                                                    [2145]
                                190                                                    [2039]
                                200                                                    [2014]
...                                                                                       ...
20190329 YNDX    S       2320.0 490                                                   [53495]
                                491                                                   [53486]
                                498                                                   [53477]
                                500         [53456, 53463, 53464, 53466, 53467, 53470, 534...
                         2323.0 100                                            [53450, 53455]

[35454 rows x 1 columns]

In [9]:
%%time
# добавляем flag - признак наличия более 1 элемента в группе
# добавляем seq_len - количество элементов в группе
groups['flag'] = [1 if len(groups.ID[i]) > 1 else 0 for i in range(len(groups.index))]
groups['seq_len'] = [len(groups.ID[i]) for i in range(len(groups.index))]

groups

CPU times: user 2.92 s, sys: 9.98 ms, total: 2.93 s
Wall time: 2.94 s


ID  \
DATE     SECCODE BUYSELL PRICE  VOLUME_INI                                                      
20190304 ALRS    B       93.5   40                                                     [2060]   
                                50                                                     [2015]   
                                180                                                    [2145]   
                                190                                                    [2039]   
                                200                                                    [2014]   
...                                                                                       ...   
20190329 YNDX    S       2320.0 490                                                   [53495]   
                                491                                                   [53486]   
                                498                                                   [53477]   
                                500         [53456, 53463, 53464, 53466, 53467, 53470, 534...   
                         2323.0 100                                            [53450, 53455]   

                                            flag  seq_len  
DATE     SECCODE BUYSELL PRICE  VOLUME_INI                 
20190304 ALRS    B       93.5   40             0        1  
                                50             0        1  
                                180            0        1  
                                190            0        1  
                                200            0        1  
...                                          ...      ...  
20190329 YNDX    S       2320.0 490            0        1  
                                491            0        1  
                                498            0        1  
                                500            1       10  
                         2323.0 100            1        2  

[35454 rows x 3 columns]

In [41]:
# создаем новый датафрейм, в котором будут только группы с количеством элементов > 1
# groups_1 - список элементов из groups, в состав которых вошло более одного элемента
groups_1 = groups[groups.flag == 1]

groups_1

ID  \
DATE     SECCODE BUYSELL PRICE  VOLUME_INI                                                      
20190304 ALRS    B       93.5   290                                              [1975, 2055]   
                                510                                              [2061, 2149]   
                                580                                              [2027, 2147]   
                                590                                  [2131, 2132, 2150, 2151]   
                                700                                              [2139, 2141]   
...                                                                                       ...   
20190329 YNDX    B       2295.0 100                                     [53498, 53500, 53501]   
                         2318.0 100                                            [53469, 53473]   
                         2320.0 100                                     [53451, 53452, 53471]   
                 S       2320.0 500         [53456, 53463, 53464, 53466, 53467, 53470, 534...   
                         2323.0 100                                            [53450, 53455]   

                                            flag  seq_len  
DATE     SECCODE BUYSELL PRICE  VOLUME_INI                 
20190304 ALRS    B       93.5   290            1        2  
                                510            1        2  
                                580            1        2  
                                590            1        4  
                                700            1        2  
...                                          ...      ...  
20190329 YNDX    B       2295.0 100            1        3  
                         2318.0 100            1        2  
                         2320.0 100            1        3  
                 S       2320.0 500            1       10  
                         2323.0 100            1        2  

[8216 rows x 3 columns]

# (12.6) Создание цепочек

In [17]:
# создаем новый датафрейм a на основании iceberg_all, в котором будет добавлено новое поле GROUP_ID
# поле GROUP_ID - идентификатор цепочки, в который вошла айсберг-заявка

a = []

data = iceberg_all
# data - список всех выявленных айсбергов
# groups_1 - результаты группировки, отобранные по условию, что в группу должно входить больше 1 элемента

for j in range (len(groups_1.ID)):
    
    for i in range (len(groups_1.ID[j])):
        
        string = groups_1.ID[j][i] # индентификатор первого элемента
        
        if i !=0: # если не первый элемент в списке
            
            prev_string = groups_1.ID[j][i-1] # индентификатор предыдущего элемента
            diff =  data.TIME.iloc[string] - data.TIME_IDENT.iloc[prev_string] # расчет разницы между временем выявления предыдущего айсберга и выставлением текущего
            
            if diff.seconds < 0.999999: # если разница между временем выставления текущего айсберга и временем выявления предыдущего меньше N, то сохраняется предыдущий group_id
                
                a.append({'BUYSELL': data.BUYSELL.iloc[string],
                         'DATE': data.DATE.iloc[string],
                         'ICEBERG': data.ICEBERG.iloc[string],
                         'ORDERNO': data.ORDERNO.iloc[string],
                         'PRICE': data.PRICE.iloc[string],
                         'SECCODE': data.SECCODE.iloc[string],
                         'TIME': data.TIME.iloc[string],
                         'TIME_IDENT': data.TIME_IDENT.iloc[string],
                         'VOLUME': data.VOLUME.iloc[string],
                         'VOLUME_INI': data.VOLUME_INI.iloc[string],
                         'DELTA': data.DELTA.iloc[string],
                         'ID': data.ID.iloc[string],
                         'GROUP_ID': group_id,
                            'Path_numb':j})
            
            else: # если разница между временем выставления текущего айсберга и временем выявления предыдущего не меньше N, то определяется новый group_id
                
                group_id = groups_1.ID[j][i] # индентификатор цепочки, в которую вошла айсберг-заявка
                a.append({'BUYSELL': data.BUYSELL.iloc[string],
                         'DATE': data.DATE.iloc[string],
                         'ICEBERG': data.ICEBERG.iloc[string],
                         'ORDERNO': data.ORDERNO.iloc[string],
                         'PRICE': data.PRICE.iloc[string],
                         'SECCODE': data.SECCODE.iloc[string],
                         'TIME': data.TIME.iloc[string],
                         'TIME_IDENT': data.TIME_IDENT.iloc[string],
                         'VOLUME': data.VOLUME.iloc[string],
                         'VOLUME_INI': data.VOLUME_INI.iloc[string],
                         'DELTA': data.DELTA.iloc[string],
                         'ID': data.ID.iloc[string],
                         'GROUP_ID': group_id,
                            'Path_numb':j})
        
        else: # если это первая запись в группе, то определяется новый group_id
            
            group_id = groups_1.ID[j][i] # индентификатор цепочки, в которую вошла айсберг-заявка
            a.append({'BUYSELL': data.BUYSELL.iloc[string],
                         'DATE': data.DATE.iloc[string],
                         'ICEBERG': data.ICEBERG.iloc[string],
                         'ORDERNO': data.ORDERNO.iloc[string],
                         'PRICE': data.PRICE.iloc[string],
                         'SECCODE': data.SECCODE.iloc[string],
                         'TIME': data.TIME.iloc[string],
                         'TIME_IDENT': data.TIME_IDENT.iloc[string],
                         'VOLUME': data.VOLUME.iloc[string],
                         'VOLUME_INI': data.VOLUME_INI.iloc[string],
                         'DELTA': data.DELTA.iloc[string],
                         'ID': data.ID.iloc[string],
                         'GROUP_ID': group_id,
                            'Path_numb':j})
                

In [18]:
# преобразуем в датафрейм
a = pd.DataFrame(a)

a

,BUYSELL,DATE,ICEBERG,ORDERNO,PRICE,SECCODE,TIME,TIME_IDENT,VOLUME,VOLUME_INI,DELTA,ID,GROUP_ID,Path_numb
0,B,20190304,1,1996506,93.5,ALRS,2019-03-04 16:42:56.501831,2019-03-04 16:42:57.045617,-2000,290,0 days 00:00:00.543786000,1975,1975,0
1,B,20190304,1,2011649,93.5,ALRS,2019-03-04 16:47:33.793267,2019-03-04 16:47:34.450499,-470,290,0 days 00:00:00.657232000,2055,2055,0
2,B,20190304,1,2011748,93.5,ALRS,2019-03-04 16:47:36.309530,2019-03-04 16:47:36.543418,-100,510,0 days 00:00:00.233888000,2061,2061,1
3,B,20190304,1,2478082,93.5,ALRS,2019-03-04 18:18:43.865644,2019-03-04 18:18:45.010659,-410,510,0 days 00:00:01.145015000,2149,2149,1
4,B,20190304,1,2010605,93.5,ALRS,2019-03-04 16:47:13.621744,2019-03-04 16:47:14.153135,-270,580,0 days 00:00:00.531391000,2027,2027,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27263,S,20190329,1,2205983,2320.0,YNDX,2019-03-29 16:15:45.266712,2019-03-29 16:25:55.007902,-1009,500,0 days 00:10:09.741190000,53480,53480,8214
27264,S,20190329,1,2283797,2320.0,YNDX,2019-03-29 16:30:29.191404,2019-03-29 16:30:29.284633,-14,500,0 days 00:00:00.093229000,53496,53496,8214
27265,S,20190329,1,2284252,2320.0,YNDX,2019-03-29 16:30:31.451968,2019-03-29 16:30:40.610152,-32,500,0 days 00:00:09.158184000,53497,53497,8214
27266,S,20190329,1,537474,2323.0,YNDX,2019-03-29 11:03:07.752503,2019-03-29 11:05:02.797062,-17,100,0 days 00:01:55.044559000,53450,53450,8215


In [19]:
a.loc[(a['ID']==test[0]) | 
      (a['ID']==test[1]) | 
      (a['ID']==test[2]) | 
      (a['ID']==test[3]) |
      (a['ID']==test[4]) |
      (a['ID']==test[5]) |
      (a['ID']==test[6]) |
      (a['ID']==test[7]) |
      (a['ID']==test[8]) |
      (a['ID']==test[9]) ]

,BUYSELL,DATE,ICEBERG,ORDERNO,PRICE,SECCODE,TIME,TIME_IDENT,VOLUME,VOLUME_INI,DELTA,ID,GROUP_ID,Path_numb
27256,S,20190329,1,2060992,2320.0,YNDX,2019-03-29 15:42:53.893738,2019-03-29 15:42:53.893738,-61,500,0 days 00:00:00.000000000,53456,53456,8214
27257,S,20190329,1,2061002,2320.0,YNDX,2019-03-29 15:42:53.899903,2019-03-29 15:42:53.901716,-208,500,0 days 00:00:00.001813000,53463,53456,8214
27258,S,20190329,1,2061028,2320.0,YNDX,2019-03-29 15:42:53.909352,2019-03-29 15:42:53.913124,-16,500,0 days 00:00:00.003772000,53464,53456,8214
27259,S,20190329,1,2065192,2320.0,YNDX,2019-03-29 15:43:21.560966,2019-03-29 15:43:32.587501,-1,500,0 days 00:00:11.026535000,53466,53466,8214
27260,S,20190329,1,2067595,2320.0,YNDX,2019-03-29 15:43:55.342162,2019-03-29 15:44:12.641059,-406,500,0 days 00:00:17.298897000,53467,53467,8214
27261,S,20190329,1,2084710,2320.0,YNDX,2019-03-29 15:47:57.422170,2019-03-29 15:49:33.071498,-232,500,0 days 00:01:35.649328000,53470,53470,8214
27262,S,20190329,1,2128127,2320.0,YNDX,2019-03-29 16:00:06.165129,2019-03-29 16:00:24.768060,-291,500,0 days 00:00:18.602931000,53474,53474,8214
27263,S,20190329,1,2205983,2320.0,YNDX,2019-03-29 16:15:45.266712,2019-03-29 16:25:55.007902,-1009,500,0 days 00:10:09.741190000,53480,53480,8214
27264,S,20190329,1,2283797,2320.0,YNDX,2019-03-29 16:30:29.191404,2019-03-29 16:30:29.284633,-14,500,0 days 00:00:00.093229000,53496,53496,8214
27265,S,20190329,1,2284252,2320.0,YNDX,2019-03-29 16:30:31.451968,2019-03-29 16:30:40.610152,-32,500,0 days 00:00:09.158184000,53497,53497,8214


In [155]:
a.sort_values(by=['ID']).head(50)

,BUYSELL,DATE,ICEBERG,ORDERNO,PRICE,SECCODE,TIME,TIME_IDENT,VOLUME,VOLUME_INI,DELTA,ID,GROUP_ID,Path_numb
327,B,20190304,1,693576,155.83,GAZP,2019-03-04 11:21:49.469445,2019-03-04 11:34:25.806855,-3000,2000,0 days 00:12:36.337410000,8,8,95
329,B,20190304,1,914063,155.99,GAZP,2019-03-04 12:06:42.051496,2019-03-04 12:06:42.051496,-3000,1000,0 days 00:00:00.000000000,16,16,96
330,B,20190304,1,914064,155.99,GAZP,2019-03-04 12:06:42.051496,2019-03-04 12:06:42.051496,-750,1000,0 days 00:00:00.000000000,17,16,96
328,B,20190304,1,926050,155.83,GAZP,2019-03-04 12:08:27.916832,2019-03-04 12:20:31.420411,-3000,2000,0 days 00:12:03.503579000,23,23,95
322,B,20190304,1,1110466,155.55,GAZP,2019-03-04 12:47:35.569858,2019-03-04 12:47:35.569858,-2440,1000,0 days 00:00:00.000000000,33,33,94
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25861,B,20190329,1,3805366,92.47,ALRS,2019-03-29 18:30:59.945237,2019-03-29 18:31:00.407356,-220,1000,0 days 00:00:00.462119000,54497,54497,7776
25862,B,20190329,1,3821189,92.47,ALRS,2019-03-29 18:32:18.319863,2019-03-29 18:32:19.963461,-560,1000,0 days 00:00:01.643598000,54498,54498,7776
25885,B,20190329,1,3845898,92.58,ALRS,2019-03-29 18:35:14.255473,2019-03-29 18:35:14.256529,-1000,1000,0 days 00:00:00.001056000,54502,54502,7782
25886,B,20190329,1,3845899,92.58,ALRS,2019-03-29 18:35:14.255473,2019-03-29 18:35:14.256529,-490,1000,0 days 00:00:00.001056000,54503,54503,7782


# (12.7) Оценка истинного объема заявки

In [61]:
%%time
# группируем айсберг-заявки из датафрейма a по GROUP_ID и Path_numb
# for_big_ice - результаты группировки айсберг-заявок по идентификатору цепочки айсберг-заявок
for_big_ice = a.groupby(['GROUP_ID','Path_numb'])['ID'].count()

for_big_ice

GROUP_ID  Path_numb
8         95           1
16        96           2
23        95           1
33        94           1
34        93           2
                      ..
54497     7776         1
54498     7776         1
54502     7782         1
54503     7782         1
54505     7784         1
Name: ID, Length: 18734, dtype: int64

In [62]:
# преобразуем в датафрейм
for_big_ice = pd.DataFrame(for_big_ice)

for_big_ice

,,ID
GROUP_ID,Path_numb,
8,95,1
16,96,2
23,95,1
33,94,1
34,93,2
...,...,...
54497,7776,1
54498,7776,1
54502,7782,1


In [105]:
# отбираем только те группы, в которые вошло больше 1 элемента
# big_ice - список цепочек из for_big_ice, в состав которых вошло более одной айсберг-заявки
big_ice = for_big_ice[for_big_ice.ID != 1]

big_ice

,,ID
GROUP_ID,Path_numb,
16,96,2
34,93,2
37,94,2
40,92,5
45,94,2
...,...,...
54440,7773,2
54453,7777,2
54459,7771,2


In [122]:
%%time
# создаем новый датафрейм b на основании a, в который будут добавлены новые поля: 
# TOTAL_VOLUME - общий объем цепочки
# VOLUME_LEFT - объем, который остался в айсберге после вскрытия текущего элемента цепочки айсберга

b = []

for i in range(len(big_ice)):
    
    ind = big_ice.index[i][0] # инентификатор цепочки
    test_2 = a.loc[(a.GROUP_ID == ind)] # датафрейм с айсберг-заявками, включенными в цепочку
    sum_vol = 0
     
    for j in range(len(test_2.ID)): # считаем общий объем цепочки
        
        sum_vol = sum_vol + (test_2.VOLUME_INI.iloc[j] - test_2.VOLUME.iloc[j])
    
    vol_left = sum_vol
    
    for k in range(len(test_2.ID)): # считаем оставшийся объем цепочки
        
        vol_left = vol_left - (test_2.VOLUME_INI.iloc[k] - test_2.VOLUME.iloc[k])
        id =  test_2.ID.iloc[k] # ID элемента
        string = a.index[a['ID'] == id][0] # индекс строки, в которой содержится элемент с таким ID
    
        b.append({'BUYSELL': a.BUYSELL.iloc[string],
                      'DATE': a.DATE.iloc[string],
                      'ICEBERG': a.ICEBERG.iloc[string],
                      'ORDERNO': a.ORDERNO.iloc[string],
                      'PRICE': a.PRICE.iloc[string],
                      'SECCODE': a.SECCODE.iloc[string],
                      'TIME': a.TIME.iloc[string],
                      'TIME_IDENT': a.TIME_IDENT.iloc[string],
                      'VOLUME': a.VOLUME.iloc[string],
                      'VOLUME_INI': a.VOLUME_INI.iloc[string],
                      'DELTA': a.DELTA.iloc[string],
                      'ID': a.ID.iloc[string],
                      'GROUP_ID': a.GROUP_ID.iloc[string],
                      'Path_numb':a.Path_numb.iloc[string],
                      'TOTAL_VOLUME': sum_vol,
                        'VOLUME_LEFT': vol_left })
    
b = pd.DataFrame(b)
b

CPU times: user 13.5 s, sys: 117 ms, total: 13.6 s
Wall time: 14 s


,BUYSELL,DATE,ICEBERG,ORDERNO,PRICE,SECCODE,TIME,TIME_IDENT,VOLUME,VOLUME_INI,DELTA,ID,GROUP_ID,Path_numb,TOTAL_VOLUME,VOLUME_LEFT
0,B,20190304,1,914063,155.99,GAZP,2019-03-04 12:06:42.051496,2019-03-04 12:06:42.051496,-3000,1000,0 days 00:00:00.000000000,16,16,96,5750,1750
1,B,20190304,1,914064,155.99,GAZP,2019-03-04 12:06:42.051496,2019-03-04 12:06:42.051496,-750,1000,0 days 00:00:00.000000000,17,16,96,5750,0
2,B,20190304,1,1110468,155.55,GAZP,2019-03-04 12:47:35.571816,2019-03-04 12:47:35.603881,-440,560,0 days 00:00:00.032065000,34,34,93,1600,600
3,B,20190304,1,1110474,155.55,GAZP,2019-03-04 12:47:35.631700,2019-03-04 12:47:37.802127,-40,560,0 days 00:00:02.170427000,35,34,93,1600,0
4,B,20190304,1,1117143,155.55,GAZP,2019-03-04 12:48:34.302065,2019-03-04 12:48:36.038058,-470,1000,0 days 00:00:01.735993000,37,37,94,3270,1800
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9386,B,20190329,1,1202856,93.11,ALRS,2019-03-29 12:53:51.411988,2019-03-29 12:53:51.411988,-480,1000,0 days 00:00:00.000000000,54201,54200,7803,6390,0
9387,B,20190329,1,1206034,93.00,ALRS,2019-03-29 12:54:51.910077,2019-03-29 12:55:08.066349,-5690,1000,0 days 00:00:16.156272000,54204,54204,7799,9870,3180
9388,B,20190329,1,1206688,93.00,ALRS,2019-03-29 12:55:08.129376,2019-03-29 12:55:08.129376,-2180,1000,0 days 00:00:00.000000000,54205,54204,7799,9870,0
9389,B,20190329,1,3712745,92.27,ALRS,2019-03-29 18:20:46.659777,2019-03-29 18:21:00.149496,-360,1000,0 days 00:00:13.489719000,54487,54487,7766,3220,1860
